In [2]:
import polars as pl
import datetime as dt
import numpy as np

In [63]:
def compute_weights(covariance, expected_returns):
    return (lambda x: x/np.sum(x))(np.linalg.solve(covariance, expected_returns))


def weights_error(cov_est, cov_real, expected_returns):
    return np.linalg.norm(compute_weights(cov_est, expected_returns) - compute_weights(cov_real, expected_returns))


In [64]:
start_date = dt.date(1996,7,3) #min value 1996,7,3
end_date = dt.date(2024, 12, 31) #max value 2024, 12, 31

bab = pl.read_parquet("signal_bab.parquet").filter(pl.col("date").is_between(start_date, end_date)) #1
meanrev = pl.read_parquet("signal_meanrev.parquet").filter(pl.col("date").is_between(start_date, end_date)) #2
momentum = pl.read_parquet("signal_momentum.parquet").filter(pl.col("date").is_between(start_date, end_date)) #3

expected_returns_data = pl.read_parquet("qMatrix.parquet").filter(pl.col("date").is_between(start_date, end_date))

weekday = list(map(lambda date : date.weekday(), bab.select(pl.col("date").alias("date")).to_series().to_list())) 
returns1 = bab.select(pl.col("bab_return").alias("return1")).to_numpy()
returns2 = meanrev.select(pl.col("meanrev_return").alias("return2")).to_numpy()
returns3 = momentum.select(pl.col("momentum_return").alias("return3")).to_numpy()

len = returns1.size

returns = [np.array([returns1[i], returns2[i], returns3[i]]) for i in range(0, len)]

expected_returns = expected_returns_data.select(
    pl.col("bab_er").alias("er1"),
    pl.col("meanrev_er").alias("er2"),
    pl.col("momentum_er").alias("er3")
).to_numpy()

cov_matrix_real = [(np.outer((returns[i] - expected_returns[i]), (returns[i] - expected_returns[i]))) for i in range(0, len)]

def recurse(x_new, x_prev, day, r, c) : #Converges to the geometrically weighted mean.
    if (day == 0): return (1-r)*x_new + (c*x_new + (1-c)*x_prev)*(r + r*r) + r*r*r*x_prev #If the day is monday, we use an alternative formula because weekends are non-trading days.
    else: return (1-r)*x_new + r*x_prev

r = 0.9
c = 0.5

#Remember, the estimated covariance matrix is the expected covariance matrix for the next day.
cov_matrix_est = [cov_matrix_real[0]]
for i in range(1, len-1):
    cov_matrix_est.append(recurse(cov_matrix_real[i], cov_matrix_est[i-1], weekday[i], r, c))

error = [weights_error(cov_est=cov_matrix_est[i], cov_real=cov_matrix_real[i+1], expected_returns=expected_returns[i]) for i in range(1, len - 1)]


print(np.average(error))



ValueError: solve1: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (m,m),(m)->(m) (size 3 is different from 9)

In [10]:
signal_data = pl.read_parquet("signal_data.parquet")
weights = pl.read_parquet("updated_weights.parquet")

In [12]:
print(weights)

shape: (20_764_025, 5)
┌────────────┬─────────┬─────────────┬────────────────┬─────────────────┐
│ date       ┆ barrid  ┆ bab_weight  ┆ meanrev_weight ┆ momentum_weight │
│ ---        ┆ ---     ┆ ---         ┆ ---            ┆ ---             │
│ date       ┆ str     ┆ f64         ┆ f64            ┆ f64             │
╞════════════╪═════════╪═════════════╪════════════════╪═════════════════╡
│ 1995-06-30 ┆ USAA191 ┆ -1.0844e-20 ┆ null           ┆ null            │
│ 1995-06-30 ┆ USAA1W1 ┆ -9.1201e-22 ┆ null           ┆ null            │
│ 1995-06-30 ┆ USAA1Y1 ┆ -4.8273e-21 ┆ null           ┆ null            │
│ 1995-06-30 ┆ USAA2L1 ┆ -4.2673e-21 ┆ null           ┆ null            │
│ 1995-06-30 ┆ USAA311 ┆ -9.2198e-21 ┆ null           ┆ null            │
│ …          ┆ …       ┆ …           ┆ …              ┆ …               │
│ 2025-09-15 ┆ USBQZR1 ┆ -6.9890e-22 ┆ -1.3553e-20    ┆ null            │
│ 2025-09-15 ┆ USBRGT1 ┆ -3.7165e-21 ┆ -3.4045e-19    ┆ null            │
│ 2025-09-15 ┆ 

In [ ]:
signal_weights = pl.read_parquet("../signal_weights/weights/all_weights_pivot.parquet")

In [13]:
print(signal_weights)

shape: (20_764_025, 5)
┌────────────┬─────────┬─────────────┬────────────────┬─────────────────┐
│ date       ┆ barrid  ┆ bab_weight  ┆ meanrev_weight ┆ momentum_weight │
│ ---        ┆ ---     ┆ ---         ┆ ---            ┆ ---             │
│ date       ┆ str     ┆ f64         ┆ f64            ┆ f64             │
╞════════════╪═════════╪═════════════╪════════════════╪═════════════════╡
│ 1995-06-30 ┆ USAA191 ┆ -1.0844e-20 ┆ null           ┆ null            │
│ 1995-06-30 ┆ USAA1W1 ┆ -9.1201e-22 ┆ null           ┆ null            │
│ 1995-06-30 ┆ USAA1Y1 ┆ -4.8273e-21 ┆ null           ┆ null            │
│ 1995-06-30 ┆ USAA2L1 ┆ -4.2673e-21 ┆ null           ┆ null            │
│ 1995-06-30 ┆ USAA311 ┆ -9.2198e-21 ┆ null           ┆ null            │
│ …          ┆ …       ┆ …           ┆ …              ┆ …               │
│ 2025-09-15 ┆ USBQZR1 ┆ -6.9890e-22 ┆ -6.1406e-20    ┆ null            │
│ 2025-09-15 ┆ USBRGT1 ┆ -3.7165e-21 ┆ -6.0722e-19    ┆ null            │
│ 2025-09-15 ┆ 